In [2]:
import os
from tensorflow import keras

basepath = os.path.abspath(os.path.join(os.curdir, ".."))
weights_path = os.path.join(
    basepath, "tf_models", "cifair10_weigths.h5")

def make_model(input_shape):
    "Create a model with batch normalization."
    layers = [keras.layers.Flatten(input_shape=input_shape)]
    for _ in range(20):
        layers.append(keras.layers.BatchNormalization())
        layers.append(keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal"))
    layers.append(keras.layers.BatchNormalization())
    layers.append(keras.layers.Dense(10, activation="softmax"))

    model = keras.models.Sequential(layers)

    model.compile(
        loss="sparse_categorical_crossentropy",
        optimizer=keras.optimizers.Nadam(),
        metrics=["accuracy"]
    )

    model.save_weights(weights_path)

    return model

In [3]:
model = make_model((32, 32, 3))

In [4]:
model_path = os.path.join(basepath, "tf_models", "cifair10_model.h5")
model.save(model_path)

In [5]:
from keras.datasets import cifar10

def load_cifair10():
    "Import CIFAIR10 dataset and split into train, validation and test sets."
    data = cifar10.load_data()
    (X_train_full, y_train_full), (X_test, y_test) = data
    X_valid, X_train = X_train_full[:5000] / 255.0, X_train_full[5000:] / 255.0
    y_valid, y_train = y_train_full[:5000], y_train_full[5000:]
    X_test = X_test / 255.0
    
    return (X_train, y_train, X_valid, y_valid, X_test, y_test)

In [6]:
import h5py

(X_train, y_train, X_val, y_val, X_test, y_test) = load_cifair10()
filepath = os.path.join(basepath, "tf_models", "cifair10.h5")

with h5py.File(filepath, "w") as f:
    f.create_dataset("X_train", data=X_train)
    f.create_dataset("y_train", data=y_train)

    f.create_dataset("X_val", data=X_val)
    f.create_dataset("y_val", data=y_val)

    f.create_dataset("X_test", data=X_test)
    f.create_dataset("y_test", data=y_test)